In [1]:
import sys
import platform
import os

# sys.path.append(pth)
# from Utils.OneHotGen import *
os.chdir("/Users/saksh.menon/Documents/GitHub/C-RNN-approach")

from Utils.dataframe_gen import *
from Utils.OneHot_gen import *
from Utils.TFIDF_gen import *
from Utils.word2vec_gen import *
import numpy as np


In [2]:
df = dataframe_init()
w2v = word2vec_vectorize(df)

2024-02-26 03:11:06.855425: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-26 03:11:06.855448: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-02-26 03:11:06.855452: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-02-26 03:11:06.855477: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-26 03:11:06.855490: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-02-26 03:11:06.948223: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/Users/saksh.menon/Documents/Gi

(218,)